<a href="https://colab.research.google.com/github/SubEdit/LookMovie-DL/blob/master/CHI_Map_image_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install GDAL and required Python packages
!apt-get install -y gdal-bin
!pip install pillow requests

In [54]:
#@title Initiate Modules
import os
import math
import requests
from PIL import Image

# Functions to convert between lat/lon and tile numbers
def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + 1/math.cos(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)  # returns (lat, lon)

# Define bounding box for Harar, Ethiopia.
# Adjust these values as needed.
area_of_interest = (9.292, 42.08, 9.333, 42.16)  # (lat_min, lon_min, lat_max, lon_max)
lat_min, lon_min, lat_max, lon_max = area_of_interest

zoom = 18
# Compute tile range.
# For the upper-left of the area, use the maximum latitude and minimum longitude.
# For the lower-right, use the minimum latitude and maximum longitude.
x_min, y_max = deg2num(lat_max, lon_min, zoom)  # top-left tile
x_max, y_min = deg2num(lat_min, lon_max, zoom)  # bottom-right tile

print("Tile X range: {} to {}".format(x_min, x_max))
print("Tile Y range: {} to {}".format(y_min, y_max))

# Create directory to store downloaded tiles
tile_dir = "tiles"
os.makedirs(tile_dir, exist_ok=True)

tile_size = 256  # standard tile size in pixels

Tile X range: 161713 to 161771
Tile Y range: 124275 to 124245


In [ ]:
# @title Area Input Form (Default- Harar)

lat_min = 9.292  # @param {type:"number"}
lon_min = 42.08  # @param {type:"number"}
lat_max = 9.333  # @param {type:"number"}
lon_max = 42.16  # @param {type:"number"}
zoom = 18  # @param {type:"slider", min:1, max:18}

area_of_interest = (lat_min, lon_min, lat_max, lon_max)

print("Area of Interest:", area_of_interest)
print("Zoom Level:", zoom)

In [56]:
import shutil
shutil.rmtree('tiles')
os.mkdir('tiles')
if y_min > y_max:
    y_min, y_max = y_max, y_min
if x_min > x_max:
    x_min, x_max = x_max, x_min
x_min, x_max, y_min, y_max
# https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/16/31072/40443

In [ ]:
#@title Run to Download each tile from ArcGIS World Imagery.
# Note: The ArcGIS service returns JPEG images.
for x in range(x_min, x_max + 1):
    for y in range(y_min, y_max + 1):
        url = f"https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{zoom}/{y}/{x}"
        tile_filename = os.path.join(tile_dir, f"{zoom}_{x}_{y}.jpg")
        if not os.path.exists(tile_filename):
            print("Downloading", url)
            r = requests.get(url)
            if r.status_code == 200:
                with open(tile_filename, "wb") as f:
                    f.write(r.content)
            else:
                print("Failed to download", url)


In [59]:
#@title Run to Stitch tiles together using PIL. (download merged_tiles.jpg when done)
num_tiles_x = x_max - x_min + 1
num_tiles_y = y_max - y_min + 1
merged_img = Image.new('RGB', (num_tiles_x * tile_size, num_tiles_y * tile_size))

# In the tile scheme, y increases downward.
for i, x in enumerate(range(x_min, x_max + 1)):
    for j, y in enumerate(range(y_min, y_max + 1)):
        tile_path = os.path.join(tile_dir, f"{zoom}_{x}_{y}.jpg")
        try:
            tile = Image.open(tile_path)
            merged_img.paste(tile, (i * tile_size, j * tile_size))
        except Exception as e:
            print("Error opening", tile_path, e)

merged_filename = "merged_tiles.jpg"
merged_img.save(merged_filename)
print("Merged image saved as", merged_filename)

Merged image saved as merged_tiles.jpg
